In [2]:
%load_ext autoreload
%autoreload 2
import os
import glob
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
pd.options.display.max_columns = 999
#pd.options.display.max_rows = 100
import settings as s
import images as im
import seaborn as sns 
import indicoio
import sys
import utils
sys.path.append("../Tools/python-client")
from __future__ import print_function
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

# Betaface

In [54]:
beta_api_key = 'd45fd466-51e2-4701-8da8-04351c872236'

In [82]:
def get_betaface_features(f_list):
    beta_feats = ['age', 'gender', 'attractive', 'young', 'race']
    feats = []
    for f in f_list:
        converted = json.loads(str(f).replace('\'','\"'))
        if converted['name'] in beta_feats:
            feats.append([converted['name'], converted['value'], converted['confidence']])
    return feats


In [153]:
def get_betaface(img_dir): # create an instance of the API class
    api_instance = swagger_client.MediaApi()
     # str | your API key or d45fd466-51e2-4701-8da8-04351c872236
    try:
        # upload media file using multipart/form-data
        api_response = api_instance.v2_media_file_post(beta_api_key, img_dir)
        num_faces = len(api_response.media.faces) if api_response.media.faces is not None else 0
        if num_faces != 1:
            print(str(num_faces) + " people detected.")
            return None
        else:
            return get_betaface_features(api_response.media.faces[0].tags)
    except ApiException as e:
        print(e)
        return None

In [89]:
a = get_betaface(os.path.join(s.SCRAPER_DIR, 'img/sample/94475.jpg'))

In [144]:
beta_keys = ['age', 'attractive', 'gender','race' , 'young']

In [156]:
def create_betaface_table(imgs):
    people = []
    for i in imgs:
        res = get_betaface(i)
        if res is not None:
            people.append([os.path.basename(i).split('.')[0]]+[(r[1],r[2]) for r in res] )
        else:
            people.append([os.path.basename(i).split('.')[0]]+[None for k in beta_keys])
    return pd.DataFrame(people, columns=['name']+beta_keys)
    

# Cognitive API

In [14]:
sys.path.append("../Cognitive-Face-Python/")
import cognitive_face as CF
KEY = '92663da6b71b4c7cac8fee67bb283097'  # Replace with a valid subscription key (keeping the quotes in place).
CF.Key.set(KEY)
BASE_URL = 'https://uksouth.api.cognitive.microsoft.com/face/v1.0'  # Replace with your regional Base URL
CF.BaseUrl.set(BASE_URL)

In [165]:
def get_cognitive(img_dir): # create an instance of the API class
    api_instance = swagger_client.MediaApi()
     # str | your API key or d45fd466-51e2-4701-8da8-04351c872236
    try:
        resp = CF.face.detect(img_dir, attributes=['age,gender,smile,noise'])
        num_faces = len(resp) if resp is not None else 0
        if num_faces != 1:
            print(str(num_faces) + " people detected.")
            return None
        else:
            return resp[0]['faceAttributes']
    except Exception as e:
        print(e)
        return None

In [100]:
b = get_cognitive(os.path.join(s.SCRAPER_DIR, 'img/sample/94475.jpg'))

/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [101]:
cognitive_keys = ['age','gender','smile','noise']

In [104]:
",".join(cognitive_keys)

'age,gender,smile,noise'

In [174]:
def create_cognitive_table(imgs):
    people = []
    for i, im in zip(range(len(imgs)), imgs):
        res = get_cognitive(im)
        if res is not None:
            people.append([os.path.basename(im).split('.')[0]]+[res[k] if k in res.keys() else None for k in cognitive_keys ])
        else:
            people.append([os.path.basename(im).split('.')[0]]+[None for k in cognitive_keys])
        if (i+1) % 20 == 0:
            time.sleep(60)
    return pd.DataFrame(people, columns=['name']+cognitive_keys)

# Sightcorp

In [43]:
import requests
import json
sight_api_key = 'cc5dd55921a64f2e822c677c8d1b727b'

In [178]:
def get_sightcorp(img_dir): 
    try:
        json_resp = requests.post( 'https://api-face.sightcorp.com/api/detect/',
              data  = { 'app_key' : sight_api_key, 'ethnicity' : True },
              files = { 'img'     : ( 'filename', open( img_dir, 'rb' ) ) } )
        people = json.loads(json_resp.text.replace('\n', ''))['people']
        sight_keys = ['age', 'gender', 'emotions', 'ethnicity']
        num_faces = len(people) if people is not None else 0
        if num_faces != 1:
            print(str(num_faces) + " people detected.")
            return None
        else:
            return {key:people[0][key] for key in sight_keys}
    except Exception as e:
        print(e)
        return None

In [136]:
get_sightcorp(os.path.join(s.SCRAPER_DIR, 'img/sample/94475.jpg'))

{'age': 48,
 'emotions': {'anger': 2,
  'disgust': 3,
  'fear': 1,
  'happiness': 76,
  'sadness': 7,
  'surprise': 2},
 'ethnicity': {'african': 47, 'asian': 2, 'caucasian': 43, 'hispanic': 7},
 'gender': 86}

In [180]:
sight_keys = ['age', 'gender', 'emotions', 'ethnicity']

In [183]:
def create_sightcorp_table(imgs):
    people = []
    for i in imgs:
        res = get_sightcorp(i)
        if res is not None:
            people.append([os.path.basename(i).split('.')[0]]+[res[k] if k in res.keys() else None for k in sight_keys ])
        else:
            people.append([os.path.basename(i).split('.')[0]]+[None for k in sight_keys])
        time.sleep(1)
    return pd.DataFrame(people, columns=['name']+sight_keys)

# Setup Experiment

In [148]:
img_files = glob.glob(os.path.join(s.SCRAPER_DIR + '/img/sample/*'))[::-1]

In [157]:
beta_results = create_betaface_table(img_files)

0 people detected.
0 people detected.


In [158]:
beta_results

,name,age,attractive,gender,race,young
0,160529866,"(46, 0.6)","(no, 0.22)","(female, 0.55)","(white, 0.6)","(yes, 0.42)"
1,171004104,"(23, 0.6)","(yes, 0.6)","(female, 0.95)","(hispanic, 0.83)","(yes, 1.0)"
2,85725,"(23, 0.6)","(no, 1.0)","(male, 0.96)","(white, 0.5)","(no, 0.66)"
3,161570593,None,None,None,None,None
4,84116,"(54, 0.6)","(no, 0.93)","(male, 0.65)","(white, 1.0)","(no, 0.87)"
5,1147347,"(18, 0.6)","(no, 1.0)","(male, 0.47)","(white, 0.65)","(no, 0.17)"
6,184634330,"(20, 0.6)","(no, 0.33)","(female, 0.85)","(asian, 0.99)","(yes, 0.89)"
7,876370,"(22, 0.6)","(no, 0.95)","(male, 0.52)","(asian, 0.99)","(no, 0.21)"
8,94475,"(36, 0.6)","(no, 0.06)","(female, 0.81)","(white, 0.99)","(no, 0.15)"
9,181492157,"(27, 0.6)","(no, 0.46)","(female, 0.95)","(black, 0.55)","(yes, 0.2)"


In [175]:
cognitive_results = create_cognitive_table(img_files)

/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urlli

0 people detected.


/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urlli

0 people detected.


/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/urlli

In [176]:
cognitive_results

,name,age,gender,smile,noise
0,160529866,27.0,female,1.000,"{'noiseLevel': 'low', 'value': 0.0}"
1,171004104,24.0,female,0.444,"{'noiseLevel': 'low', 'value': 0.0}"
2,85725,24.0,female,0.001,"{'noiseLevel': 'low', 'value': 0.11}"
3,161570593,21.0,female,0.001,"{'noiseLevel': 'medium', 'value': 0.47}"
4,84116,53.0,female,0.998,"{'noiseLevel': 'low', 'value': 0.0}"
5,1147347,NaN,None,NaN,None
6,184634330,25.0,female,0.400,"{'noiseLevel': 'high', 'value': 0.76}"
7,876370,27.0,female,1.000,"{'noiseLevel': 'low', 'value': 0.09}"
8,94475,44.0,female,0.993,"{'noiseLevel': 'low', 'value': 0.0}"
9,181492157,32.0,female,1.000,"{'noiseLevel': 'medium', 'value': 0.65}"


In [184]:
sightcorp_results = create_sightcorp_table(img_files)

0 people detected.
0 people detected.
0 people detected.
0 people detected.
0 people detected.
0 people detected.
0 people detected.


In [187]:
pd.set_option('display.max_colwidth', -1)
sightcorp_results

,name,age,gender,emotions,ethnicity
0,160529866,35.0,54.0,"{'happiness': 94, 'surprise': 1, 'anger': 0, 'disgust': 0, 'fear': 0, 'sadness': 2}","{'african': 96, 'asian': 0, 'caucasian': 2, 'hispanic': 0}"
1,171004104,24.0,87.0,"{'happiness': 15, 'surprise': 2, 'anger': 11, 'disgust': 27, 'fear': 12, 'sadness': 14}","{'african': 9, 'asian': 4, 'caucasian': 82, 'hispanic': 3}"
2,85725,36.0,-58.0,"{'happiness': 3, 'surprise': 10, 'anger': 36, 'disgust': 10, 'fear': 18, 'sadness': 14}","{'african': 11, 'asian': 2, 'caucasian': 75, 'hispanic': 9}"
3,161570593,31.0,91.0,"{'happiness': 45, 'surprise': 6, 'anger': 10, 'disgust': 15, 'fear': 5, 'sadness': 13}","{'african': 5, 'asian': 4, 'caucasian': 88, 'hispanic': 1}"
4,84116,60.0,65.0,"{'happiness': 38, 'surprise': 6, 'anger': 10, 'disgust': 4, 'fear': 5, 'sadness': 1}","{'african': 6, 'asian': 0, 'caucasian': 90, 'hispanic': 2}"
5,1147347,NaN,NaN,None,None
6,184634330,33.0,77.0,"{'happiness': 41, 'surprise': 5, 'anger': 6, 'disgust': 19, 'fear': 8, 'sadness': 15}","{'african': 79, 'asian': 1, 'caucasian': 17, 'hispanic': 0}"
7,876370,NaN,NaN,None,None
8,94475,48.0,86.0,"{'happiness': 76, 'surprise': 2, 'anger': 2, 'disgust': 3, 'fear': 1, 'sadness': 7}","{'african': 47, 'asian': 2, 'caucasian': 43, 'hispanic': 7}"
9,181492157,31.0,88.0,"{'happiness': 99, 'surprise': 0, 'anger': 0, 'disgust': 0, 'fear': 0, 'sadness': 0}","{'african': 73, 'asian': 0, 'caucasian': 23, 'hispanic': 1}"
